# Pathology prediction (PNEUMONIA)


@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the pneumonia pathology using the CSV embeddings file

## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples :  0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value. 

The model is based on binary classification for each target chest pathology.

Final sample size for the Pneumonia pathology is : (N =  7225)

#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)


#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.PNEUMONIA,
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="Pneumonia", experiment_name="CP_Pneumonia", filepath="./results")

# Model training and results evaluation
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='stratifiedkfold',
    outer_fold=5,
    outer_strategy='stratifiedkfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-09-27 17:42:40,522	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=678263) Outer fold 1
(run_fold pid=678263) Configuring PyCaret for outer fold 1
(run_fold pid=678258) Outer fold 2


Processing:  75%|███████▌  | 3/4 [04:58<01:50, 110.38s/it] [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(run_fold pid=678258)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=678258) Fold                                                          
(run_fold pid=678258) 0       0.7535  0.8412  0.7789  0.7634  0.7711  0.5042  0.5043
(run_fold pid=678258) 1       0.7968  0.8702  0.8357  0.7938  0.8142  0.5902  0.5912
(run_fold pid=678258) 2       0.8076  0.8898  0.8215  0.8182  0.8198  0.6133  0.6133
(run_fold pid=678258) 3       0.8065  0.8876  0.8089  0.8240  0.8164  0.6119  0.6120
(run_fold pid=678258) 4       0.7987  0.8789  0.8435  0.7920  0.8169  0.5939  0.5953
(run_fold pid=678258) Mean    0.7926  0.8735  0.8177  0.7983  0.8077  0.5827  0.5832
(run_fold pid=678258) Std     0.0200  0.0176  0.0227  0.0216  0.0184  0.0403  0.0404
(run_fold pid=678258) Configuring PyCaret for outer fold 2
(run_fold pid=678258) Tuning hyperparameters for model xgboost with custom grid using grid search


                                                          
Processing:  75%|███████▌  | 3/4 [05:02<01:51, 111.85s/it]


...

(run_fold pid=678263)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=678263) Fold                                                          
(run_fold pid=678263) 0       0.8000  0.8672  0.8337  0.7996  0.8163  0.5970  0.5977
(run_fold pid=678263) 1       0.7795  0.8587  0.7911  0.7943  0.7927  0.5571  0.5571
(run_fold pid=678263) 2       0.7892  0.8676  0.8211  0.7906  0.8056  0.5755  0.5760
(run_fold pid=678263) 3       0.7870  0.8728  0.8069  0.7956  0.8012  0.5719  0.5720
(run_fold pid=678263) 4       0.7933  0.8766  0.8232  0.7957  0.8092  0.5838  0.5842
(run_fold pid=678263) Mean    0.7898  0.8686  0.8152  0.7952  0.8050  0.5771  0.5774
(run_fold pid=678263) Std     0.0068  0.0060  0.0148  0.0029  0.0079  0.0132  0.0134
(run_fold pid=678263) Tuning hyperparameters for model xgboost with custom grid using grid search


(run_fold pid=678258) Transformation Pipeline and Model Successfully Saved
(run_fold pid=678258)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=678258) 0  Extreme Gradient Boosting    0.8194  0.8958  ...  0.8326  0.6366  0.6368
(run_fold pid=678258) 
(run_fold pid=678258) [1 rows x 8 columns]
(run_fold pid=678258) Outer fold 5
(run_fold pid=678258) Configuring PyCaret for outer fold 5


Processing:  75%|███████▌  | 3/4 [04:54<01:49, 109.16s/it]


(run_fold pid=678258)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=678258) Fold                                                          
(run_fold pid=678258) 0       0.7643  0.8667  0.7972  0.7691  0.7829  0.5254  0.5258
(run_fold pid=678258) 1       0.7849  0.8718  0.8114  0.7905  0.8008  0.5670  0.5673
(run_fold pid=678258) 2       0.8076  0.8769  0.8333  0.8103  0.8216  0.6128  0.6131
(run_fold pid=678258) 3       0.7827  0.8697  0.8008  0.7928  0.7968  0.5633  0.5634
(run_fold pid=678258) 4       0.7684  0.8509  0.7886  0.7791  0.7838  0.5344  0.5345
(run_fold pid=678258) Mean    0.7816  0.8672  0.8063  0.7883  0.7972  0.5606  0.5608
(run_fold pid=678258) Std     0.0152  0.0088  0.0154  0.0139  0.0141  0.0307  0.0307
(run_fold pid=678258) Tuning hyperparameters for model xgboost with custom grid using grid search


(run_fold pid=678263) Transformation Pipeline and Model Successfully Saved
(run_fold pid=678263)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=678263) 0  Extreme Gradient Boosting    0.8048  0.8925  ...  0.8224  0.6063  0.6076
(run_fold pid=678263) 
(run_fold pid=678263) [1 rows x 8 columns]
(run_fold pid=678258) Transformation Pipeline and Model Successfully Saved
(run_fold pid=678258)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=678258) 0  Extreme Gradient Boosting    0.7979  0.8916  ...  0.8119  0.5936  0.5937
(run_fold pid=678258) 
(run_fold pid=678258) [1 rows x 8 columns]
Final metrics table:
     Metric     Mean   Std Dev
0  Accuracy  0.79220  0.014893
1       AUC  0.87688  0.012199
2    Recall  0.81380  0.016299
3     Prec.  0.79992  0.014935
4        F1  0.80676  0.013924
5     Kappa  0.58210  0.029971
6       MCC  0.58230  0.029977
